In [1]:
import fastrepl
from fastrepl.llama_index import FastREPLCallbackHandler

ds = fastrepl.Dataset.from_dict({})

evaluator = fastrepl.RAGEvaluator(
    node=fastrepl.RAGAS(model="gpt-3.5-turbo", metric="Faithfulness")
)

fastrepl_callback_handler = FastREPLCallbackHandler(dataset=ds)

In [2]:
from llama_index import ServiceContext
from llama_index.callbacks import CallbackManager

callback_manager = CallbackManager([fastrepl_callback_handler])
service_context = ServiceContext.from_defaults(callback_manager=callback_manager)

In [3]:
from llama_index import SimpleWebPageReader

urls = ["http://paulgraham.com/greatwork.html"]
documents = SimpleWebPageReader(html_to_text=True).load_data(urls)
assert len(documents) == 1

In [4]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(service_context=service_context)

In [5]:
questions = [
    "just say anything you want",
    "How does the author suggest that one can determine if a field is suited for them?",
    "How does the author recommend ensuring that something created for others is something they actually want?",
]

_ = fastrepl.local_runner(fn=query_engine.query).run(
    args_list=[(q,) for q in questions]
)

Output()

In [6]:
ds

fastrepl.Dataset({
    features: ['question', 'contexts', 'answer', 'elapsed'],
    num_rows: 3
})

In [7]:
result = fastrepl.local_runner(evaluator=evaluator, dataset=ds).run()
result

Output()

fastrepl.Dataset({
    features: ['question', 'contexts', 'answer', 'elapsed', 'result'],
    num_rows: 3
})

In [8]:
result["result"], result["elapsed"]

([0.33333333333333337, 1.0, 0.6666666666666667],
 [8.294839, 8.909655, 9.418836])